In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cairns,AU,2022-03-31 14:23:31,-16.9167,145.7667,71.55,82,18,0.00
1,1,Pompton Lakes,US,2022-03-31 14:23:32,41.0054,-74.2907,58.28,92,100,1.99
2,2,Mataura,NZ,2022-03-31 14:23:32,-46.1927,168.8643,46.63,93,100,2.89
3,3,Borgarnes,IS,2022-03-31 14:23:32,64.5383,-21.9206,42.66,62,96,1.41
4,4,Rikitea,PF,2022-03-31 14:23:32,-23.1203,-134.9692,77.70,74,82,16.62


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Rikitea,PF,2022-03-31 14:23:32,-23.1203,-134.9692,77.70,74,82,16.62
5,5,Pangai,TO,2022-03-31 14:23:33,-19.8000,-174.3500,81.52,78,100,12.03
7,7,Bathsheba,BB,2022-03-31 14:23:34,13.2167,-59.5167,75.81,83,75,10.36
8,8,Darab,IR,2022-03-31 14:23:34,28.7519,54.5444,82.00,11,23,5.99
14,14,Arraial Do Cabo,BR,2022-03-31 14:23:40,-22.9661,-42.0278,85.96,74,0,12.66
21,21,Buchanan,LR,2022-03-31 14:23:42,5.8808,-10.0467,85.39,68,66,8.68
22,22,Butaritari,KI,2022-03-31 14:23:42,3.0707,172.7902,79.83,81,57,11.90
23,23,Hithadhoo,MV,2022-03-31 14:23:43,-0.6000,73.0833,83.46,69,7,5.41
26,26,Padang,ID,2022-03-31 14:23:48,-0.9492,100.3543,77.99,84,8,3.27
27,27,Kavaratti,IN,2022-03-31 14:23:49,10.5669,72.6420,83.66,74,5,6.58


In [16]:
preferred_cities_df.count()

City_ID       163
City          163
Country       163
Date          163
Lat           163
Lng           163
Max Temp      163
Humidity      163
Cloudiness    163
Wind Speed    163
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Rikitea,PF,77.70,-23.1203,-134.9692,
5,Pangai,TO,81.52,-19.8000,-174.3500,
7,Bathsheba,BB,75.81,13.2167,-59.5167,
8,Darab,IR,82.00,28.7519,54.5444,
14,Arraial Do Cabo,BR,85.96,-22.9661,-42.0278,
21,Buchanan,LR,85.39,5.8808,-10.0467,
22,Butaritari,KI,79.83,3.0707,172.7902,
23,Hithadhoo,MV,83.46,-0.6000,73.0833,
26,Padang,ID,77.99,-0.9492,100.3543,
27,Kavaratti,IN,83.66,10.5669,72.6420,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}